<img src="https://s3.amazonaws.com/datascienceheroes.com/EDV/360_banner_python.png" width="400px">


# Escuela de Datos Vivos 

## LAB P.D.3.1-3.2: Introducción al one hot encoding / dummy vars
Creado por Pablo Casas | https://escueladedatosvivos.ai   

Resuelto por Pablo Sotomayor

In [1]:
# Librerías que utilizaremos

import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from funpymodeling.exploratory import status 
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

## Ejercicios

1) Cargar los datos de 'data/hotels_smp.csv' (fuente: [tidytuesday](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-02-11/readme.md))

2) Separar en training (TR) y testing (TS).

3) Discretizar las variables categóricas usando TR (`get_dummies`), y aplicar la discretización en TS. Dentro del TS, modifiquen "a mano" una de las variables categóricas para que haya un valor nuevo. 

4) Parte fundamental del data scientist es inspeccionar que las transformaciones, y los flujos de datos sean los esperados. Por tal motivo debe auditarse el resultado: Revisar que la transformación sea la correcta. 

## Solución

#### 1) Carga de datos

In [2]:
data = pd.read_csv('../data/hotels_smp.csv', sep=',')

print(f"Cantidad de registros: {data.shape[0]}")
print(f"Cantidad de variables: {data.shape[1]}")

Cantidad de registros: 3000
Cantidad de variables: 32


Inspeccionamos los datos tomando una muestra aleatoria de ellos.

In [3]:
data.sample(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
2464,Resort Hotel,0,176,2016,July,30,19,1,5,2,...,No Deposit,240.0,NaN,0,Transient,193.67,1,2,Check-Out,2016-07-25
275,City Hotel,1,152,2016,June,26,19,2,3,2,...,No Deposit,8.0,NaN,0,Transient,108.12,0,0,Canceled,2016-03-21
1519,City Hotel,1,35,2017,August,32,8,0,1,2,...,No Deposit,229.0,NaN,0,Transient-Party,90.00,0,0,Canceled,2017-07-07
2331,Resort Hotel,0,234,2016,September,37,9,1,2,2,...,No Deposit,NaN,NaN,0,Transient-Party,77.03,0,1,Check-Out,2016-09-12
1680,City Hotel,0,118,2016,June,24,6,1,2,1,...,No Deposit,191.0,NaN,0,Transient-Party,110.00,0,0,Check-Out,2016-06-09


Revisamos el estado de los datos

In [4]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,hotel,0,0.000000,0,0.000000,2,object
1,is_canceled,0,0.000000,1901,0.633667,2,int64
2,lead_time,0,0.000000,147,0.049000,392,int64
3,arrival_date_year,0,0.000000,0,0.000000,3,int64
4,arrival_date_month,0,0.000000,0,0.000000,12,object
5,arrival_date_week_number,0,0.000000,0,0.000000,53,int64
6,arrival_date_day_of_month,0,0.000000,0,0.000000,31,int64
7,stays_in_weekend_nights,0,0.000000,1327,0.442333,9,int64
8,stays_in_week_nights,0,0.000000,152,0.050667,19,int64
9,adults,0,0.000000,13,0.004333,4,int64


Vimos en un proyecto que anterior que la variable 'reservation_status' no da información, pues coincide con los registros de la variable a predecir. La variable "reservation_status_date" tampoco da información relevante, también la eliminamos. Hay variables categóricas mal declaradas como numéricas: "is_canceled", "is_repeated_guest", y "babies". Hacemos las transformaciones a los datos.

In [5]:
data_original = data.copy()

data = data.drop(['reservation_status', 'reservation_status_date'], axis=1)

data['is_canceled'] = data['is_canceled'].astype('str')
data['is_canceled'] = data['is_canceled'].replace('0', 'no').replace('1', 'yes') 

data['is_repeated_guest'] = data['is_repeated_guest'].astype('str')
data['is_repeated_guest'] = data['is_repeated_guest'].replace('0', 'no').replace('1', 'yes')    

data['babies'] = data['babies'].astype('str')
data['babies'] = data['babies'].replace('0', 'no').replace('1', 'yes') 

#### 2) Determinación de los conjuntos de entrenamiento (TR) y de testeo (TS)

Establecemos las variables independientes y la variable a predecir

In [6]:
x_data, y_data = data.drop('is_canceled', axis=1), data['is_canceled']

Dividimos los datos en TR y TS. Elegimos un tamaño de 20% para los datos de testeo y adoptamos una semilla para reproducir los resultados.

In [7]:
x_tr, x_ts, y_tr, y_ts = train_test_split(x_data, y_data, test_size=0.2, random_state=0)

Juntamos los datos que corresponden a TR y TS

In [8]:
TR, TS = pd.concat([x_tr, y_tr], axis=1), pd.concat([x_ts, y_ts], axis=1)

Vimos que la variable "deposit_type" tiene las clases muy desbalanceadas. Revisamos que valores se encuentran en TR y TS

In [9]:
TR['deposit_type'].value_counts()

No Deposit    2104
Non Refund     294
Refundable       2
Name: deposit_type, dtype: int64

In [10]:
TS['deposit_type'].value_counts()

No Deposit    528
Non Refund     72
Name: deposit_type, dtype: int64

Por lo tanto, en TR hay valores que TS "no ha visto", lo cual es ideal para nuestro problema.

#### 3) Discretizamos las variables categóricas en TR

Pasamos primero todas las variables a numéricas con el método one-hot encoding. Agregamos una categoría para los valores nulos y eliminamos la redundancia en las entradas.

In [11]:
TR_ohe = pd.get_dummies(TR, drop_first=True, dummy_na=True)

Este método genera nuevos datos innecesarios para el caso de registros que no tenían variables nulas. Eliminamos esos nuevos atributos.

In [12]:
vars_to_drop = status(TR_ohe).query("unique==1")['variable']

TR_ohe = TR_ohe.drop(vars_to_drop, axis=1)

Inspecciamos los datos en TR

In [13]:
TR_ohe.head(5)

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,previous_cancellations,previous_bookings_not_canceled,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,is_canceled_yes
2370,30,2016,22,23,1,1,2,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1774,9,2016,18,29,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
731,135,2016,40,25,2,0,3,0,0,0,...,0,0,0,0,0,0,0,1,0,0
271,26,2016,47,17,0,2,2,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1077,0,2016,31,29,0,1,2,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Revisamos las etiquetas de los atributos

In [14]:
TR_ohe.columns.to_list()

['lead_time',
 'arrival_date_year',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'agent',
 'company',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'hotel_Resort Hotel',
 'arrival_date_month_August',
 'arrival_date_month_December',
 'arrival_date_month_February',
 'arrival_date_month_January',
 'arrival_date_month_July',
 'arrival_date_month_June',
 'arrival_date_month_March',
 'arrival_date_month_May',
 'arrival_date_month_November',
 'arrival_date_month_October',
 'arrival_date_month_September',
 'babies_yes',
 'meal_FB',
 'meal_HB',
 'meal_SC',
 'meal_Undefined',
 'country_ARE',
 'country_ARG',
 'country_ARM',
 'country_AUS',
 'country_AUT',
 'country_BDI',
 'country_BEL',
 'country_BGR',
 'country_BRA',
 'country_CHE',
 'country_CHL',
 'country_CHN',
 'country_C

Si deseamos guardar el encoding para usarlo posteriormente en producción utilizamos la librería pickle

In [15]:
with open('categories_TR_ohe.pickle', 'wb') as handle:
    pickle.dump(TR_ohe.columns, handle, protocol=pickle.HIGHEST_PROTOCOL)

Para cargar el encoding usamos la misma librería

In [16]:
with open('categories_TR_ohe.pickle', 'rb') as handle:
    TR_ohe_col = pickle.load(handle)

Elegimos una de las variables categóricas en el TS para que haya un valor nuevo (es decir, que no fue entrenado en el modelo). 

Variable elegida: deposit_type, agregamos el dato "Partial Refund". Cambiamos el 1% de los datos a ese valor. Los índices de los datos que transformaremos son elegidos al azar con una semilla para reproducir los resultados

In [17]:
np.random.seed(40)

indexs = np.random.choice(TS.index.to_list(), size=int(0.1*TS.shape[0]), replace=False)
TS['deposit_type'].update(pd.Series(len(indexs)*['Partial Refund'], index=indexs))
TS['deposit_type'].value_counts()

No Deposit        474
Non Refund         66
Partial Refund     60
Name: deposit_type, dtype: int64

Hacemos get_dummies al TS, con las mismas condiciones que se lo hicimos al TR.

In [18]:
TS_ohe = pd.get_dummies(TS, drop_first=True, dummy_na=True)

In [19]:
vars_to_drop = status(TS_ohe).query("unique==1")['variable']

TS_ohe = TS_ohe.drop(vars_to_drop, axis=1)

In [20]:
TS_ohe.head(5)

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,previous_cancellations,previous_bookings_not_canceled,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,deposit_type_Non Refund,deposit_type_Partial Refund,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,is_canceled_yes
311,230,2016,37,6,0,5,2,0,0,0,...,0,0,0,0,0,1,0,0,1,1
1025,182,2017,17,23,2,2,2,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1587,243,2015,39,20,2,2,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2941,260,2016,39,22,0,3,2,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2980,165,2016,29,11,3,7,2,0,0,0,...,0,0,0,0,0,0,0,1,0,1


Pero queremos entrenar a nuestro modelo con las mismas variables que el TR, para esto usamos la función reindex

In [21]:
TS_ohe_TRcols = pd.get_dummies(TS_ohe).reindex(columns=TR_ohe_col)

In [22]:
TS_ohe_TRcols.head(5)

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,previous_cancellations,previous_bookings_not_canceled,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,is_canceled_yes
311,230,2016,37,6,0,5,2,0,0,0,...,0,0,0,0,0,NaN,0,0,1,1
1025,182,2017,17,23,2,2,2,0,0,0,...,0,0,0,0,0,NaN,0,1,0,1
1587,243,2015,39,20,2,2,2,0,0,0,...,0,0,0,0,0,NaN,0,0,1,0
2941,260,2016,39,22,0,3,2,0,0,0,...,0,0,0,0,0,NaN,0,0,1,1
2980,165,2016,29,11,3,7,2,0,0,0,...,0,0,0,0,0,NaN,0,1,0,1


In [23]:
TS_ohe_TRcols["deposit_type_Refundable"].value_counts()

Series([], Name: deposit_type_Refundable, dtype: int64)

El atributo "deposit_type_Refundable" es una categoría faltante previo a haber hecho el one-hot encoding al TS. Lo reemplazamos por ceros.

In [24]:
TS_ohe_TRcols = pd.get_dummies(TS_ohe).reindex(columns=TR_ohe_col, fill_value=0)

In [25]:
TS_ohe_TRcols.head(5)

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,previous_cancellations,previous_bookings_not_canceled,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,is_canceled_yes
311,230,2016,37,6,0,5,2,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1025,182,2017,17,23,2,2,2,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1587,243,2015,39,20,2,2,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2941,260,2016,39,22,0,3,2,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2980,165,2016,29,11,3,7,2,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [26]:
TS_ohe_TRcols['deposit_type_Refundable'].value_counts()

0    600
Name: deposit_type_Refundable, dtype: int64

Vemos que se ha solucionado el problema.

In [27]:
TS_ohe_TRcols['deposit_type_Non Refund'].value_counts()

0    534
1     66
Name: deposit_type_Non Refund, dtype: int64